In [93]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA


In [94]:
model = pickle.load(open('/home/smayan/Desktop/DJS-compute-tasks/DJS-compute-tasks/Task 2/Data/model.sav', 'rb'))

# Load the dataset with accurate categories and columns (forstreamlit.csv)
df_forstreamlit = pd.read_csv('/home/smayan/Desktop/DJS-compute-tasks/DJS-compute-tasks/Task 2/Notebook/forstreamlit.csv')

# Define categorical and numerical columns based on the dataset
nonnumeric = ['make', 'fuel-type', 'aspiration', 'body-style', 'drive-wheels',
              'engine-location', 'engine-type', 'fuel-system']
numerical_columns = ['wheel-base', 'length', 'width', 'curb-weight', 
                     'engine-size', 'bore', 'horsepower', 'city-mpg', 'highway-mpg']

In [95]:
df_forstreamlit.head(5)

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,0,3,121.132075,alfa-romero,gas,std,2.0,convertible,rwd,front,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,1,3,121.132075,alfa-romero,gas,std,2.0,convertible,rwd,front,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,2,1,121.132075,alfa-romero,gas,std,2.0,hatchback,rwd,front,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,3,2,164.000000,audi,gas,std,4.0,sedan,fwd,front,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,4,2,164.000000,audi,gas,std,4.0,sedan,4wd,front,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [96]:
df_forstreamlit.drop(columns=['Unnamed: 0'], axis=1,inplace=True)

In [97]:
features = df_forstreamlit.drop(columns=['price'], axis=1).columns.tolist()

In [98]:
features

['symboling',
 'normalized-losses',
 'make',
 'fuel-type',
 'aspiration',
 'num-of-doors',
 'body-style',
 'drive-wheels',
 'engine-location',
 'wheel-base',
 'length',
 'width',
 'height',
 'curb-weight',
 'engine-type',
 'num-of-cylinders',
 'engine-size',
 'fuel-system',
 'bore',
 'stroke',
 'compression-ratio',
 'horsepower',
 'peak-rpm',
 'city-mpg',
 'highway-mpg']

In [99]:
pca = PCA(n_components=0.96)

In [100]:
st.title("Car Price Prediction App")

input_data = {}
for col in nonnumeric:
    unique_values = df_forstreamlit[col].unique().tolist()
    input_data[col] = st.selectbox(f"Select {col}", unique_values)

# Numerical inputs (sliders) based on actual min/max values from the DataFrame
for col in numerical_columns:
    min_val = float(df_forstreamlit[col].min())
    max_val = float(df_forstreamlit[col].max())
    input_data[col] = st.slider(f"Select {col}", min_val, max_val, (min_val + max_val) / 2)

In [101]:
input_df = pd.DataFrame([input_data], columns=df_forstreamlit.columns)

# Trial code
columns = input_df.columns

# Generate random values (0 or 1) for all columns in input_df
random_values = np.random.randint(0, 2, size=(1, len(columns)))

# Create a DataFrame with the random values
input_data = pd.DataFrame(random_values, columns=columns)


X_encoded = pd.get_dummies(input_data, columns=nonnumeric)
df_combined = pd.concat([input_data.drop(columns=nonnumeric).reset_index(allow_duplicates=False, drop=True), X_encoded.reset_index(allow_duplicates=False,drop=True)], axis=1)

X = df_combined.drop(['price'], axis = 1)
X_reduced = pca.fit_transform(scale(X_encoded))

/home/smayan/Desktop/AI-ML-DS/sample_project/env/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


In [104]:
column_names = [f'PC{i+1}' for i in range(X_reduced.shape[1])]
X_reduced_test_df = pd.DataFrame(X_reduced, columns=column_names)
X_reduced_test_df.head(5)
X_reduced_test= pca.fit_transform(scale(X))[:,:29]

/home/smayan/Desktop/AI-ML-DS/sample_project/env/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


In [107]:
prediction = model.predict(X_reduced_test)

ValueError: X has 1 features, but LinearRegression is expecting 29 features as input.